<a href="https://colab.research.google.com/github/sugarforever/LlamaIndex-Tutorials/blob/main/03_Customization/03_Customization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U llama-index chromadb

In [ ]:
!mkdir data
!wget https://raw.githubusercontent.com/jerryjliu/llama_index/main/examples/paul_graham_essay/data/paul_graham_essay.txt -O data/paul_graham_essay.txt

In [ ]:
!pip install openai

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'your valid openai api key'

import chromadb
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index import ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index import StorageContext
from llama_index.llms import OpenAI

## 自定义文档分块，指定 LLM

In [ ]:
service_context = ServiceContext.from_defaults(chunk_size=500, llm=OpenAI())


## 自定义向量存储

In [ ]:
chroma_client = chromadb.PersistentClient()
chroma_collection = chroma_client.create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

## 索引文档

In [ ]:
documents = SimpleDirectoryReader('data').load_data()
index = VectorStoreIndex.from_documents(documents, service_context=service_context,storage_context=storage_context)

## 指定响应模式，以及启用流式响应

In [ ]:
query_engine = index.as_query_engine(response_mode='tree_summarize', streaming=True)
response = query_engine.query("What did the author do?")
response.print_response_stream()